In [1]:
import torch
import random

import torch.nn as nn

/Users/luan/Developer/bow-text-classifier/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [2]:
from bow_text_classifier.data import (
    get_dataset_filepath,
    _parse_data,
    _create_dict,
    _create_tensors,
)

# with open(get_dataset_filepath("dev"), "r") as f:
#     dev_data = _parse_data(f.read())

with open(get_dataset_filepath("train"), "r") as f:
    train_data = _parse_data(f.read())

with open(get_dataset_filepath("test"), "r") as f:
    test_data = _parse_data(f.read())


word_to_index, tag_to_index = _create_dict(train_data)

word_to_index, tag_to_index = _create_dict(
    test_data, word_to_index, tag_to_index, check_unk=True
)

number_of_words = len(word_to_index)
number_of_tags = len(tag_to_index)

train_data = list(_create_tensors(train_data, word_to_index, tag_to_index))
test_data = list(_create_tensors(test_data, word_to_index, tag_to_index))

number_of_words = len(word_to_index)
number_of_tags = len(tag_to_index)

In [3]:
from bow_text_classifier.nn import BoW, sentence_to_tensor

device = "cuda" if torch.cuda.is_available() else "cpu"


type = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
out = sentence_to_tensor("i love dogs", word_to_index).type(type)
test_model = BoW(number_of_words, number_of_tags).to(device)

test_model(out)

tensor([[ 0.0174,  0.0024, -0.0141,  0.0054, -0.0260]],
       grad_fn=<ViewBackward0>)

In [4]:
# train and test the BoW model
model = BoW(number_of_words, number_of_tags).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
type = torch.LongTensor

if torch.cuda.is_available():
    model.to(device)
    type = torch.cuda.LongTensor


# perform training of the Bow model
def train_bow(model, optimizer, criterion, train_data):
    for ITER in range(10):
        # perform training
        model.train()
        random.shuffle(train_data)
        total_loss = 0.0
        train_correct = 0
        for sentence, tag in train_data:
            sentence = torch.tensor(sentence).type(type)
            tag = torch.tensor([tag]).type(type)
            output = model(sentence)
            predicted = torch.argmax(output.data.detach()).item()

            loss = criterion(output, tag)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if predicted == tag:
                train_correct += 1
        # perform testing of the model
        model.eval()
        test_correct = 0
        for _sentence, tag in test_data:
            _sentence = torch.tensor(_sentence).type(type)
            output = model(_sentence)
            predicted = torch.argmax(output.data.detach()).item()
            if predicted == tag:
                test_correct += 1

        # print model performance results
        log = (
            f"ITER: {ITER+1} | "
            f"train loss/sent: {total_loss/len(train_data):.4f} | "
            f"train accuracy: {train_correct/len(train_data):.4f} | "
            f"test accuracy: {test_correct/len(test_data):.4f}"
        )
        print(log)


# call the train_bow function
train_bow(model, optimizer, criterion, train_data)

ITER: 1 | train loss/sent: 1.4736 | train accuracy: 0.3599 | test accuracy: 0.4050
ITER: 2 | train loss/sent: 1.1217 | train accuracy: 0.6084 | test accuracy: 0.4127
ITER: 3 | train loss/sent: 0.9117 | train accuracy: 0.7100 | test accuracy: 0.4181
ITER: 4 | train loss/sent: 0.7691 | train accuracy: 0.7729 | test accuracy: 0.4127
ITER: 5 | train loss/sent: 0.6626 | train accuracy: 0.8070 | test accuracy: 0.4127
ITER: 6 | train loss/sent: 0.5811 | train accuracy: 0.8337 | test accuracy: 0.4127
ITER: 7 | train loss/sent: 0.5175 | train accuracy: 0.8526 | test accuracy: 0.4090
ITER: 8 | train loss/sent: 0.4638 | train accuracy: 0.8684 | test accuracy: 0.3995
ITER: 9 | train loss/sent: 0.4187 | train accuracy: 0.8810 | test accuracy: 0.4059
